In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import sklearn

In [2]:
# Main NLP Class

from pipeUtils import Document
from pipeUtils import Annotation
import re

class PadClassificationSystem:
    def __init__(self):
        #initiate necessary components        
        self.target_rules=self.getTargetRegexes()        
        self.negation_rules = self.getNegRegexes()
                
    def process(self, document):
        document_id = document.document_id
        ann_index=0
        for reg in self.target_rules:
            for match in reg.finditer(document.text):
                ann_id = 'NLP_'+ str(document_id) + '_' + str(ann_index)
                ann_index=ann_index+1
                new_annotation = Annotation(start_index=int(match.start()), 
                                    end_index=int(match.end()), 
                                    type='pad_annotation',
                                    ann_id = ann_id
                                    )
                new_annotation.spanned_text = document.text[new_annotation.start_index:new_annotation.end_index]

                # Check negation right before the found target up to 30 charachers before, 
                # making sure that the pre-text does not cross the text boundary and is valid

                if new_annotation.start_index - 30 > 0:
                    pre_text_start = new_annotation.start_index - 30
                else:
                    pre_text_start = 0

                # ending index of the pre_text is the beginning of the found target    
                pre_text_end = new_annotation.start_index    

                # substring the document text to identify the pre_text string
                pre_text = doc.text[pre_text_start: pre_text_end]

                # We do not need to know the exact location of the negation keyword, so re.search is acceptable
                for neg_regex in self.negation_rules:
                    if re.search(neg_regex, pre_text):
                        new_annotation.attributes["Negation"] ="Negated"

                document.annotations.append(new_annotation)
        
        return document 
    
    def getTargetRegexes(self):
        target_regexes = []
        regexes = [
            r'\bp(eri\w+)?\s*(a(rt\w+)?|v(as\w+)?)\s*d(ise\w+)?\b',
            r'\bhistory\s*(?:\w+\W+){1,5}?p(eri\w+)?\s*(a(rt\w+)?|v(as\w+)?)\s*d(ise\w+)?\b',
            r'\bpvd\b',
            r'\bhas\s*(?:\w+\W+){1,5}?p(eri\w+)?\s*(a(rt\w+)?|v(as\w+)?)\s*d(ise\w+)?\b',
            r'\bsug(es\w+)\s*(?:\w+\W+){1,5}?p(eri\w+)?\s*(a(rt\w+)?|v(as\w+)?)\s*d(ise\w+)?\b',
            r'\bdiagnosis\s*(?:\w+\W+){1,5}?p(eri\w+)?\s*(a(rt\w+)?|v(as\w+)?)\s*d(ise\w+)?\b',
            #r'\ba(nk\w+)?(\s*\-|\\|\/)?\s*b(ra\w+)?\s*i(nd\w+)?\b((\W*)?(?:\w+\W+){1,5}?(0*(\.\d{1,2}))|(1*(\.[4-9])))+'
            
        ]
        for reg in regexes:
            target_regexes.append(re.compile(reg, re.IGNORECASE))
        return target_regexes

    def getNegRegexes(self):
        target_regexes = []
        regexes = [
            r'\bno\b',
            r'no\s*evidence\s*of'  ,
            r'does\s*not\s*have',
            r'denies'
            #r'^(?!.*(\bp(eri\w+)?\s*(a(rt\w+)?|v(as\w+)?)\s*d(ise\w+)?\b)).*$',
            #r'^(?!.*(\ba(nk\w+)?(\s*\-|\\|\/)?\s*b(ra\w+)?\s*i(nd\w+)?\b)).*$'
        ]
        for reg in regexes:
            target_regexes.append(re.compile(reg, re.IGNORECASE))
        return target_regexes

In [3]:
#  test case
nlp_system = PadClassificationSystem()
doc_text = '''
Patient has peripheral artery disease. ---------- \nPatient also has PVD or peripheral vascular\ndisease or pvd . 
\n The patient does not have any peripheral artery disease 
but has peripheral arterial disease . The patient denies having peripheral vascular disease . \n 
The patient has a femoral and illiac occlusion which is suggestive of peripheral arterial disease.\nthis is just some random text, has no meaning at all
'''
doc=Document(text=doc_text, document_id='Doc1')
 
out_doc=nlp_system.process(doc)
print(out_doc.toString())

Doc1
-------

Patient has peripheral artery disease. ---------- 
Patient also has PVD or peripheral vascular
disease or pvd . 

 The patient does not have any peripheral artery disease 
but has peripheral arterial disease . The patient denies having peripheral vascular disease . 
 
The patient has a femoral and illiac occlusion which is suggestive of peripheral arterial disease.
this is just some random text, has no meaning at all

-------
NLP_Doc1_0 pad_annotation 13 38 peripheral artery disease 
NLP_Doc1_1 pad_annotation 69 72 PVD 
NLP_Doc1_2 pad_annotation 76 103 peripheral vascular
disease 
NLP_Doc1_3 pad_annotation 107 110 pvd 
NLP_Doc1_4 pad_annotation 146 171 peripheral artery disease [Negation:Negated]
NLP_Doc1_5 pad_annotation 181 208 peripheral arterial disease 
NLP_Doc1_6 pad_annotation 237 264 peripheral vascular disease [Negation:Negated]
NLP_Doc1_7 pad_annotation 340 367 peripheral arterial disease 
NLP_Doc1_8 pad_annotation 69 72 PVD 
NLP_Doc1_9 pad_annotation 107 110 pv

In [4]:
from pipeUtils import Annotation
from pipeUtils import Document
 
import os
import glob 

In [5]:
# Read all test documents
unid="u1166466"
project_1 = "PAD"
project_2 = "TEST"
path_1 = "/home/"+"u0420717"+"/BRAT/"+str(unid)+"/"+project_1
path_2 = "/home/"+"u0420717"+"/BRAT/"+str(unid)+"/"+project_2

In [6]:
test_docs=dict()
test_doc_paths = glob.glob(str(path_2+'/*.txt')) 
for d in test_doc_paths:
    doc = Document()
    #print(d)
    doc.load_document_from_file(d)
    #print(str(d[:-3])+'ann')
    doc.load_annotations_from_brat(str(d[:-3])+'ann')
    #print(os.path.basename(d))
    test_docs[os.path.basename(d)]=doc


test_docs 

{'10088_1.txt': <pipeUtils.Document at 0x7f447279c240>,
 '1234_8.txt': <pipeUtils.Document at 0x7f447279c080>,
 '24631_2.txt': <pipeUtils.Document at 0x7f447279c3c8>,
 '25879_5.txt': <pipeUtils.Document at 0x7f447279c2b0>,
 '3678_0.txt': <pipeUtils.Document at 0x7f447279c358>,
 '3830_3.txt': <pipeUtils.Document at 0x7f447279c160>,
 '3830_7.txt': <pipeUtils.Document at 0x7f447279c278>,
 '781_6.txt': <pipeUtils.Document at 0x7f447279c0f0>,
 '781_9.txt': <pipeUtils.Document at 0x7f447279c0b8>,
 '8396_4.txt': <pipeUtils.Document at 0x7f447279c390>}

In [7]:
# Processing the all notes
nlp_system = PadClassificationSystem()

for doc_id in  test_docs.keys():
    nlp_system.process(test_docs.get(doc_id))
    print(test_docs.get(doc_id))

In [8]:
tp_total = 0
fp_total = 0
fn_total = 0
tp_list_total = []
fp_list_total= []
fn_list_total = []
for doc_id in test_docs.keys():
    tp, fp, fn, tp_list, fp_list, fn_list = (test_docs.get(doc_id)).compare_types_by_span('PAD','pad_annotation', True)
    tp_total = tp_total + tp
    fp_total = fp_total + fp
    fn_total = fn_total + fn
    tp_list_total.extend(tp_list)
    fp_list_total.extend(fp_list)
    fn_list_total.extend(fn_list)

print('TP =',tp_total, 'FP =',fp_total, 'FN =',fn_total)

if tp_total > 0 :
    precision = tp_total / (tp_total + fp_total)
    print('Precision=',round(precision,3))

if tp_total > 0 :
    recall = tp_total / (tp_total + fn_total)
    print('Recall=',round(recall,3))

for a in tp_list_total:
    print(a[0].toString(),'||', a[1].toString())
for a in fp_list_total:
    print(a.toString())
for a in fn_list_total:
    print(a.toString())

TP = 0 FP = 0 FN = 0


In [9]:
tp_total = 0
fp_total = 0
fn_total = 0
tp_list_total = []
fp_list_total= []
fn_list_total = []
attributes_to_compare=[]
# To compare attributes, create a list of tuples for each pair to compare:
# attributes_to_compare.append[(A1_type, A1_att_name, A1_att_value),(A2_type, A2_att_name, A2_att_value)]
attributes_to_compare.append([('PAD', 'Negation', 'Negated','Hypthetical','Affirmed'),('pad_annotation', 'Negation', 'Negated','Hypothetical','Affirmed')])

for doc_id in test_docs.keys():
    tp, fp, fn, tp_list, fp_list, fn_list = (test_docs.get(doc_id)).\
    compare_types_by_span_and_attributes('PAD','pad_annotation', attributes_to_compare , False)
    tp_total = tp_total + tp
    fp_total = fp_total + fp
    fn_total = fn_total + fn
    tp_list_total.extend(tp_list)
    fp_list_total.extend(fp_list)
    fn_list_total.extend(fn_list)

print('TP =',tp_total, 'FP =',fp_total, 'FN =',fn_total)

if tp_total > 0 :
    precision = tp_total / (tp_total + fp_total)
    print('Precision=',round(precision,3))

if tp_total > 0 :
    recall = tp_total / (tp_total + fn_total)
    print('Recall=',round(recall,3))

#for a in tp_list_total:
#    print(a[0].toString(),'||', a[1].toString())
for a in fp_list_total:
    print(a.toString())
for a in fn_list_total:
    print(a.toString())

TP = 0 FP = 0 FN = 0


In [10]:
import pymysql
import pandas as pd
import getpass
import re

conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Enter MySQL passwd for jovyan········


In [11]:
pad_data = pd.read_sql("""SELECT subject_id, 
                      category, 
                      text FROM noteevents limit 100000 """,conn)

In [12]:
final_nlp_system = PadClassificationSystem()

In [13]:
output = []
counter = 0
for index , row in pad_data.sample(10000).iterrows():    
    doc = Document(document_id=str(row.subject_id) + '_' + str(index), text=row.text)
    final_nlp_system.process(doc)
    if(len(doc.annotations) > 0):
        i = 1
        for a in doc.annotations:
            if( a.type == 'pad_annotation'):
                neg_flag = 0
                # Switch the flag to 1 when the mention is negated
                if('definite_negated_existence' in a.attributes):
                    neg_flag=1
                ### Each row in the dictionary
                record_id  = str(row.subject_id) + '_' + str(index)+'_'+str(i)
                subject_id =  row.subject_id
                note_id = str(row.subject_id) + '_' + str(index)
                annotation_type = a.type
                snippet = doc.text[int(a.start_index): int(a.end_index)]
                out_list = [record_id, subject_id, note_id, annotation_type, \
                            a.start_index, a.end_index, \
                            snippet, neg_flag]
                output.append(out_list)
                i=i+1
                counter=counter+1
                # Print . after 10 identified records
                if counter%10 == 0:
                    print('.', end='')
        else:
            continue
        break
    

..........................

In [17]:
counter

260

In [14]:
columns=['record_id','subject_id', 'note_id', 'annotation_type', 'span_start', 'span_end', 'PAD_snippet', 'neg_flag']
result_data_frame = (pd.DataFrame(output, columns=columns))

result_data_frame.describe()
result_data_frame

,record_id,subject_id,note_id,annotation_type,span_start,span_end,PAD_snippet,neg_flag
0,4798_29456_1,4798,4798_29456,pad_annotation,1057,1060,pad,0
1,4798_29456_2,4798,4798_29456,pad_annotation,1071,1074,pad,0
2,1266_9219_1,1266,1266_9219,pad_annotation,427,430,PVD,0
3,1266_9219_2,1266,1266_9219,pad_annotation,427,430,PVD,0
4,12413_74693_1,12413,12413_74693,pad_annotation,573,576,PAD,0
5,12413_74693_2,12413,12413_74693,pad_annotation,2404,2407,PAD,0
6,12727_77361_1,12727,12727_77361,pad_annotation,189,192,PAD,0
7,12727_77361_2,12727,12727_77361,pad_annotation,1020,1023,PAD,0
8,12320_74388_1,12320,12320_74388,pad_annotation,1833,1836,pad,0
9,15472_91914_1,15472,15472_91914,pad_annotation,249,252,pad,0


In [15]:
result_data_frame.to_csv('PAD.csv', index=False)
print('Done')

Done
